In [7]:
#%% Cell 1: Imports and Setup
import os
import pandas as pd
import chardet

# List of required packages
required_packages = {
    'pandas': 'pandas',
    'matplotlib': 'matplotlib',
    'seaborn': 'seaborn',
    'chardet': 'chardet'
}

# Check and install missing packages
for package, import_name in required_packages.items():
    try:
        __import__(import_name)
    except ImportError:
        print(f"Installing {package}...")
        !pip install {package}

# Now import matplotlib and seaborn after ensuring they are installed
import matplotlib.pyplot as plt
import seaborn as sns

# Display settings for better output readability
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
pd.set_option('display.max_colwidth', None)


In [8]:
#%% Cell 2: Data Loading and Preparation
def load_data(file_path):
    """Load data with proper encoding detection"""
    try:
        return pd.read_csv(file_path)
    except UnicodeDecodeError:
        try:
            with open(file_path, 'rb') as f:
                encoding = chardet.detect(f.read(100000))['encoding']
            return pd.read_csv(file_path, encoding=encoding)
        except:
            return pd.read_csv(file_path, encoding='latin1')

# Path to your CSV
file_path = os.path.join("..", "csv_files", "extract.csv")

try:
    df = load_data(file_path)
    print("Dataset Overview:")
    print(f"Rows: {len(df)}, Columns: {len(df.columns)}")
    print("\nSample of data:")
    print(df.head())
except FileNotFoundError:
    print(f"Error: Could not find file at {file_path}")
    print("Please check the file path and ensure the CSV file exists.")
    df = None
except Exception as e:
    print(f"Error loading data: {str(e)}")
    print("Please check the file path and file format.")
    df = None



Dataset Overview:
Rows: 11712, Columns: 251

Sample of data:
   AccountNumber    AccountSource  \
0        45633.0        Cold Call   
1        45336.0        Cold Call   
2        45583.0        Cold Call   
3        45897.0  Outside Company   
4        45895.0        Cold Call   

                                                                Account_Inactive_Details__c  \
0                                                                                       NaN   
1                                             Circle K has created their own pizza program.   
2                                 The entire chain has changed to a different pizza program   
3  Circle K decided to pull the Pizza but not replace it. Kitchen wasn't supporting itself.   
4                                 The entire chain has changed to a different pizza program   

                   Account_Inactive_Reasons__c Account_Sold_By__c  \
0                                          NaN          ADAM HUNT   
1  Exis

/tmp/ipykernel_783102/3590407933.py:12: DtypeWarning: Columns (63,76,83,108,109,113,128,138,147,161,162,187,189,212,244) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path, encoding='latin1')


In [30]:
import json
import pandas as pd
import os
from pattern_builder import EnhancedPatternValidator  # Ensure this is implemented and available
import chardet  # To detect file encoding

def detect_file_encoding(file_path):
    """
    Detect the encoding of a file using chardet.
    """
    with open(file_path, "rb") as f:
        result = chardet.detect(f.read())
    return result["encoding"]

def analyze_data_and_generate_patterns(df):
    validator = EnhancedPatternValidator()
    analysis_results = []

    for column in df.columns:
        field_name = column
        inferred_type = None
        inferred_pattern = None

        # Analyze each cell in the column
        for value in df[column]:
            try:
                # Infer data type based on the value
                inferred_type = validator.infer_data_type(value)
                # Generate a pattern for the inferred type
                inferred_pattern = validator.build(inferred_type)
                break  # If we successfully infer type/pattern for one value, skip further checks
            except Exception as e:
                print(f"Error inferring type for value '{value}' in column '{field_name}': {e}")

        # Default to Text if no type is inferred
        if not inferred_type:
            inferred_type = "Text"
            inferred_pattern = validator.patterns.get("Text", r"^[A-Za-z0-9\s]+$")

        # Store the analysis result
        analysis_results.append({
            "fieldName": field_name,
            "fieldType": inferred_type,
            "generatedPattern": inferred_pattern
        })

    return analysis_results

def save_analysis_results(df, object_name):
    analysis_results = analyze_data_and_generate_patterns(df)

    # Prepare output directory
    output_dir = "generatedPatterns"
    os.makedirs(output_dir, exist_ok=True)

    # Output file
    output_file = os.path.join(output_dir, f"{object_name}_patterns.json")
    with open(output_file, "w") as json_file:
        json.dump(analysis_results, json_file, indent=4)

    print(f"Pattern generation completed. Results saved to {output_file}")

# Main Script
if __name__ == "__main__":
    csv_file = "../csv_files/extract.csv"
    object_name = os.path.splitext(os.path.basename(csv_file))[0]

    # Detect file encoding
    encoding = detect_file_encoding(csv_file)
    print(f"Detected encoding: {encoding}")

    # Read CSV with detected encoding
    try:
        df = pd.read_csv(csv_file, encoding=encoding)
        save_analysis_results(df, object_name)
    except Exception as e:
        print(f"Failed to process the file: {e}")


In [19]:
#%% Cell 4: Visualization of Results
def plot_confidence_scores(report_df):
    """Plot confidence scores for type predictions"""
    plt.figure(figsize=(12, 6))
    confidence_values = report_df['Confidence'].str.rstrip('%').astype(float)
    sns.barplot(x=confidence_values, y=report_df['Field Name'])
    plt.title('Confidence Scores for Data Type Predictions')
    plt.xlabel('Confidence (%)')
    plt.tight_layout()
    plt.show()

def plot_type_distribution(report_df):
    """Plot distribution of suggested types"""
    plt.figure(figsize=(10, 6))
    type_counts = report_df['Suggested Type'].value_counts()
    type_counts.plot(kind='bar')
    plt.title('Distribution of Suggested Data Types')
    plt.xlabel('Data Type')
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

if 'report_df' in locals():
    # Generate visualizations
    plot_confidence_scores(report_df)
    plot_type_distribution(report_df)



KeyError: 'Confidence'

<Figure size 1200x600 with 0 Axes>

In [5]:
#%% Cell 5: Pattern Matching Analysis
def analyze_pattern_matches(df, analysis_results):
    """Analyze how well each field matches its suggested pattern"""
    pattern_analysis = {}
    for field, analysis in analysis_results.items():
        if analysis.validation_pattern and field in df.columns:
            # Get non-null values
            valid_data = df[field].dropna().astype(str)
            if len(valid_data) == 0:
                continue

            # Check pattern matches
            matches = valid_data.str.match(analysis.validation_pattern, na=False)
            match_ratio = matches.mean() if len(matches) > 0 else 0

            # Store results
            pattern_analysis[field] = {
                'suggested_type': analysis.suggested_type,
                'match_ratio': match_ratio,
                'total_values': len(valid_data),
                'matching_values': int(matches.sum()),
                'non_matching_examples': valid_data[~matches].head(3).tolist()
            }

    return pd.DataFrame.from_dict(pattern_analysis, orient='index')

if df is not None and 'analysis_results' in locals():
    pattern_analysis_df = analyze_pattern_matches(df, analysis_results)
    print("\nPattern Matching Analysis:")
    print(pattern_analysis_df)




Pattern Matching Analysis:
                                       suggested_type  match_ratio  \
AccountNumber                                  Number     1.000000   
AccountSource                                    Text     1.000000   
Account_Inactive_Details__c                 Text Area     0.943318   
Account_Inactive_Reasons__c                      Text     1.000000   
Account_Sold_By__c                               Text     1.000000   
...                                               ...          ...   
of_Schools_within_3_miles__c                   Number     1.000000   
of_employees_dedicated_to_the_deli__c          Number     1.000000   
of_employees_usually_at_counter__c             Number     1.000000   
of_years_owner_s_has_owned_location__c         Number     1.000000   
of_years_owner_s_has_run_store__c              Number     1.000000   

                                        total_values  matching_values  \
AccountNumber                                   8551      

In [6]:
#%% Cell 6: Export Results
def export_results(report_df, pattern_analysis_df, output_dir="analysis_output"):
    """Export analysis results to CSV files"""
    os.makedirs(output_dir, exist_ok=True)
    
    # Export main report
    report_df.to_csv(os.path.join(output_dir, "field_type_analysis.csv"), index=False)

    # Export pattern analysis
    pattern_analysis_df.to_csv(os.path.join(output_dir, "pattern_matching_analysis.csv"))

    print(f"\nResults exported to {output_dir}/")

if all(var in locals() for var in ['report_df', 'pattern_analysis_df']):
    export_results(report_df, pattern_analysis_df)

#%% Cell 7: Interactive Pattern Testing
def test_pattern(df, field_name, pattern):
    """Test a custom pattern against a specific field"""
    if field_name not in df.columns:
        return "Field not found in dataset"
    
    valid_data = df[field_name].dropna().astype(str)
    matches = valid_data.str.match(pattern, na=False)

    return {
        'match_ratio': matches.mean() if len(matches) > 0 else 0,
        'total_values': len(valid_data),
        'matching_values': int(matches.sum()),
        'non_matching_examples': valid_data[~matches].head(3).tolist()
    }

# Example usage:
# if df is not None:
#     test_results = test_pattern(df, 'email_field', r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$')
#     print(test_results)
